In [ ]:
import pandas as pd
import os
import json
import random
from datetime import timedelta, datetime
import pytz 
from env import ori_path, mod_path, ntfy_nofi
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns',None)

In [ ]:
# read csv and set their col names
colum_names = ["time", "source", "srcAddr", "srcPort", "dstAddr", "dstPort", "proto", "sessionEndTime", "query", "answer", "transId", "rtt", "qclass", "qtype", "rcode", "aaFlag", "tcFlag", "rdFlag", "raFlag", "ttl"]
dns_df = pd.read_csv(ori_path + "custom_review/input_dns.csv", sep="\t", header=None, dtype=str)
dns_df.columns=colum_names

In [ ]:
dns_dfs_gen = pd.concat([dns_df] * 20, ignore_index=True)

In [ ]:
# test epoch to timestamp
# epoch_time = 1688147468.575549608
# date_time = datetime.fromtimestamp(int(epoch_time))
# fractional_seconds = epoch_time - int(epoch_time)
# formatted_fractional_seconds = format(fractional_seconds, ".9f")[1:]
# formatted_datetime = f"{date_time}{formatted_fractional_seconds}"
# formatted_datetime

In [ ]:
# random yyyy mm dd hh mm ss based on start end
def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))

In [ ]:
# formatted_epoch_time_with_nanoseconds to epoch
# formatted_dt to datetime
def random_datetime_with_nanoseconds(start, end):
    random_dt = random_date(start, end)
    random_dt_alpha = random_dt + timedelta(seconds=5)
    # Generate a random nanoseconds value (0 to 999,999,999)

    nanoseconds = random.randint(0, 999_999_999)
    # Combine the datetime and the nanoseconds
    formatted_dt = f"{random_dt.isoformat()}.{str(nanoseconds).zfill(9)}"
    epoch_time = random_dt.timestamp()
    epoch_time_alpha = random_dt_alpha.timestamp()
    # epoch_time_with_nanoseconds = epoch_time + (nanoseconds * 1e-9)
    
    # epoch timestamp full nanoseconds
    seconds_part = str(int(epoch_time))
    seconds_part_alpha = str(int(epoch_time_alpha))
    nanoseconds_part = format(nanoseconds, "09d")  # Format nanoseconds as a zero-padded string with 9 digits
    formatted_epoch_time_with_nanoseconds = f"{seconds_part}.{nanoseconds_part}"
    formatted_epoch_time_with_nanoseconds_alpha = f"{seconds_part_alpha}.{nanoseconds_part}"

    return formatted_dt, formatted_epoch_time_with_nanoseconds, formatted_epoch_time_with_nanoseconds_alpha

In [ ]:
start_date = datetime(2023, 7, 1)
end_date = datetime(2023, 8, 1)

In [ ]:
time_arr_dns = []
time_arr_dns_alpha = []
time_arr_datetime_dns = []
for i in range(0, 200):
    formatted_dt, epoch_time_with_nanoseconds, epoch_time_with_nanoseconds_alpha = random_datetime_with_nanoseconds(start_date, end_date)
    time_arr_dns.append(epoch_time_with_nanoseconds)
    time_arr_dns_alpha.append(epoch_time_with_nanoseconds_alpha)
    time_arr_datetime_dns.append(formatted_dt)
time_arr_dns.sort()
time_arr_dns_alpha.sort()
time_arr_datetime_dns.sort()

In [ ]:
time_df_dns = pd.DataFrame(time_arr_dns, columns=['time'])
time_df_dns['sessionEndTime'] = time_arr_dns_alpha
time_df_dns['timestamp'] = time_arr_datetime_dns
# time_df_dns_temp = pd.DataFrame()
# time_df_dns_temp['date'] = pd.to_datetime(time_df_dns['time']).dt.date
# time_df_dns_count = time_df_dns_temp.groupby('date').size().reset_index(name='time count')

In [ ]:
dns_dfs_gen.time = time_df_dns.time
dns_dfs_gen.sessionEndTime = time_df_dns.sessionEndTime
dns_dfs_gen.srcAddr = "41.179.253.229"

In [ ]:
dns_dfs_gen.to_csv(mod_path + "custom_review/input_dns_joe.csv", index=False, sep="\t", header=None)

In [ ]:
time_df_dns_temp = pd.DataFrame()
time_df_dns_temp['date'] = pd.to_datetime(time_df_dns['timestamp']).dt.date
time_df_dns_count = time_df_dns_temp.groupby('date').size().reset_index(name='dns_count')

In [ ]:
time_df_dns_count